# Get started with OpenACC

What will you learn here?

1. Open a parallel region with `#pragma acc parallel`
2. Activate loop parallelism with `#pragma acc loop`
3. Open a structured data region with `#pragma acc data`
4. Compile a code with OpenACC support

## OpenACC directives

If you have a CPU code and you want to get some parts on the GPU, you can add OpenACC directives to it.

A directive has the following structure:

<img alt="OpenACC directive" src="../../pictures/directive_acc.png" style="float:none" width="30%"/>

If we break it down, we have these elements:

- The sentinel is special instruction for the compiler. It tells it that what follows has to be interpreted as OpenACC
- The directive is the action to do. In the example, _parallel_ is the way to open a parallel region that will be offloaded to the GPU
- The clauses are "options" of the directive. In the example we want to copy some data on the GPU.
- The clause arguments give more details for the clause. In the example, we give the name of the variables to be copied

Some directives need to be opened just before a code block.
```fortran
!$acc parallel
  ! code block opened with `acc parallel` and closed with `acc end parallel`
!$acc end parallel
```

### A short example

With this example you can get familiar with how to run code cells during this session.
`%%idrrun` has to be present at the top of a code cell to compile and execute the code written inside the cell.

The content has to be a valid piece of code otherwise you will get errors.
In Fortran, if you want to run the code, you need to define the `main` function:
```fortran
program your_code
!
!
end program your_code
```

The example initializes an array of integers.

Example stored in: `../../examples/Fortran/Get_started_init_array_exercise.f90`

In [ ]:
%%idrrun
program array_initialisation
    use iso_fortran_env, only : INT32, REAL64
    implicit none

    integer(kind=INT32), parameter :: system_size = 100000
    integer(kind=INT32)            :: array(system_size)
    integer(kind=INT32)            :: i

    do i = 1, system_size
        array(i) = 2*i
    enddo

    write(0,"(a22,i3)") "value at position 21: ", array(21)    
end program array_initialisation

Now we add the support of OpenACC with `-a` option of idrrun.

To offload the computation on the GPU you have to open a parallel region with the directive `acc parallel` and define a code block which is affected.

Modify the cell below to perform this action. No clause are needed here.

Example stored in: `../../examples/Fortran/Get_started_init_array_exercise_acc.f90`

In [ ]:
%%idrrun -a
program array_initialisation
    use iso_fortran_env, only : INT32, REAL64
    use openacc
    implicit none

    integer(kind=INT32), parameter :: system_size = 100000
    integer(kind=INT32)            :: array(system_size)
    integer(kind=INT32)            :: i

    ! Modifications from here
    do i = 1, system_size
        array(i) = 2*i
    enddo

    write(0,"(a22,i3)") "value at position 21: ", array(21)    
end program array_initialisation

### Solution

Example stored in: `../../examples/Fortran/Get_started_init_array_solution_acc.f90`

In [ ]:
%%idrrun -a
program array_initialisation
    use iso_fortran_env, only : INT32, REAL64
    use openacc
    implicit none

    integer(kind=INT32), parameter :: system_size = 100000
    integer(kind=INT32)            :: array(system_size)
    integer(kind=INT32)            :: i

    !$acc parallel 
    do i = 1, system_size
        array(i) = 2*i
    enddo
    !$acc end parallel

    write(0,"(a22,i3)") "value at position 21: ", array(21)    
end program array_initialisation

We can have a look at a different behavior when the compiler is doing implicit stuff:

Example stored in: `../../examples/Fortran/Get_started_init_array_solution_acc_2.f90`

In [ ]:
%%idrrun -a
program array_initialisation
    use iso_fortran_env, only : INT32, REAL64
    use openacc
    implicit none

    integer(kind=INT32), parameter :: system_size = 100000
    integer(kind=INT32)            :: array(system_size)
    integer(kind=INT32)            :: i

    !$acc parallel

    !$acc loop
    do i = 1, system_size
        array(i) = 2*i
    enddo

    !$acc end parallel

    write(0,"(a22,i3)") "value at position 21: ", array(21)    
end program array_initialisation

### Let's analyze what happened

The following steps are printed:

1. the compiler command to generate the executable
2. the output of the command (displayed on red background)
3. the command line to execute the code
4. the output/error of the execution

We activated the verbose mode for the NVIDIA compilers for information about optimizations and OpenACC (compiler option -Minfo=all) and __strongly recommend that you do the same in your developments__.

The compiler found in the `main` function a __kernel__ (this is the name of code blocks offloaded to the GPU) and was able to generate code for GPU.
The line refers to the directive `acc parallel` included in the code.

By default NVIDIA compilers (formerly PGI) make an analysis of the parallel region and try to find:

- loops that can be parallelized
- data transfers needed
- operations like reductions
- etc

It might result in unexpected behavior since we did not write explicitly the directives to perform those actions.
Nevertheless, we decided to keep this feature on during the session since it is the default.
This is the reason you can see that a directive `acc loop` (used to activate loop parallelism on the GPU) was added implicitly to our code and a data transfer with `copyout`.

## Loops parallelism

Most of the parallelism in OpenACC (hence performance) comes from the loops in your code and especially from loops with __independent iterations__.
Iterations are independent when the results do not depend on the order in which the iterations are done.
Some differences due to non-associativity of operations in limited precision are usually OK.
You just have to be aware of that problem and decide if it is critical.

Another condition is that the runtime needs to know the number of iterations.
So keep incrementing integers!

### Directive

The directive to parallelize loops is:
```fortran
!$acc loop
```

### Non independent loops

Here are some cases where the iterations are not independent:

- Infinite loops
```fortran
do while(error > tolerance)
    !compute error
enddo
```

- Current iteration reads values computed by previous iterations
```fortran
array(1) = 0
array(2) = 1
do i = 3, system_size
    array(i) = array(i-1) + array(i-2)
enddo
```

- Current iteration reads values that will be changed by subsequent iterations
```fortran
do i = 1, system_size - 1
    array(i) = array(i+1) + 1
enddo
```

- Current iteration writes values that will be read by subsequent iterations
```fortran
do i= 1, system_size - 1
    array(i)   = array(i) + 1
    array(i+1) = array(i) + 2
enddo
```

These kind of loops can be offloaded to the GPU but might not give correct results if not run in sequential mode.
You can try to modify the algorithm to transform them into independent loop:

- Use temporary arrays
- Modify the order of the iterations
- etc

## Managing data in compute regions

During the porting of your code the data on which you work in the _compute regions_ might have to go back and forth between the host and the GPU.
This is important to minimize the number of data transfers because of the cost of these operations.

For each _compute region_ (i.e. `acc parallel` directive or _kernel_) a _data region_ is created.
OpenACC gives you several clauses to manage efficiently the transfers.

```fortran
!$acc parallel copy(var1(first_index:last_index)) copyin(var2(first_index_i:last_index_i,first_index_j:last_index_j), var3) copyout(var4, var5)
```

| clause      | effect when entering the region                                               | effect when leaving the region                                          |
|-------------|-------------------------------------------------------------------------------|-------------------------------------------------------------------------|
| create      | Allocate the memory needed on the GPU                                         | Free the memory on the GPU                                              |
| copyin      | Allocate the memory and initialize the variable with the values it has on CPU | Free the memory on the GPU                                              |
| copyout     | Allocate the memory needed on the GPU                                         | Copy the values from the GPU to the CPU then free the memory on the GPU |
| copy        | Allocate the memory and initialize the variable with the values it has on CPU | Copy the values from the GPU to the CPU then free the memory on the GPU |
| present     | Check if data is present: an error is raised if it is not the case            | None                                                                    |

<img alt="Data clauses" src="../../pictures/data_clauses.png" style="float:none" width="30%"/>

To choose the right data clause you need to answer the following questions:

- Does the kernel need the values computed on the host beforehand? (Before)
- Are the values computed inside the kernel needed on the host afterhand? (After)

|                  | Needed after        | Not needed after  |
|------------------|---------------------|-------------------|
|Needed Before     |  copy(var1, ...)    | copyin(var2, ...) |
|Not needed before |  copyout(var3, ...) | create(var4, ...) |

Usually it is not mandatory to specify the clauses.
The compiler will analyze your code to guess what the best solution and will tell you that one operation was done implicitely.
As a good pratice, we recommend to make all implicit operations explicit.

## Exercise: Gaussian blurring filter

In this exercise, we read a picture, load it on the GPU and then we apply a blur filter. For each pixel, the value is computed as the weighted sum of the 24 neighbors and itself with the stencil shown below:

The original picture is stored in the pictures directory. We have to convert it to RAW before loading it.

In [ ]:
import os
picture = os.path.join("..", "..", "pictures", "midris.jpg")
from idrcomp import convert_jpg_to_raw
convert_jpg_to_raw(picture, "pic.rgb")

<img alt="Stencil for Gaussian Blur" src="../../pictures/stencil_tp_blur.png" style="float:none"/>

Note: In Fortran the weights are adjusted because we do not have unsigned integers.

Your job is to offload the blur function.
Make sure that you use the correct data clauses for "pic" and "blurred" variables.

The original picture is 2232x4000 pixels.
We need 1 value for each RGB channel it means that the actual size of the matrix is 4000x12000 (3x4000).

Example stored in: `../../examples/Fortran/blur_simple_exercise.f90`

In [ ]:
%%idrrun -a
MODULE pixels
    USE OPENACC
    IMPLICIT NONE
    CONTAINS
        SUBROUTINE read_matrix_from_file(FILENAME, PIC, ROWS, COLS)
            IMPLICIT NONE
            CHARACTER(LEN=*), INTENT(IN)                :: FILENAME
            INTEGER(kind=1), DIMENSION(0:), INTENT(OUT) :: PIC
            INTEGER, INTENT(IN)                         :: ROWS, COLS
        
            INTEGER                                     :: FILE_UNIT, I, TOTAL_SIZE, IO_STATUS
            INTEGER(KIND=8)                             :: READ_SIZE
            CHARACTER(LEN=80)                           :: IO_MSG
            OPEN(NEWUNIT=FILE_UNIT, FILE=FILENAME, FORM='UNFORMATTED', ACCESS='stream')
        
            TOTAL_SIZE = ROWS * COLS * 3
            READ(FILE_UNIT, IOSTAT=IO_STATUS, IOMSG=IO_MSG) PIC(0:TOTAL_SIZE-1)
        
            CLOSE(FILE_UNIT)
        END SUBROUTINE read_matrix_from_file  

        SUBROUTINE blur(pic, blurred, rows, cols, passes)
            INTEGER(kind=1),DIMENSION(0:),INTENT(IN)  :: pic
            INTEGER(kind=1),DIMENSION(0:),INTENT(OUT) :: blurred
            INTEGER,DIMENSION(0:4,0:4)                :: coefs
            INTEGER,INTENT(IN)                        :: rows,cols, passes
            INTEGER(kind=4)                           :: i,j,p,i_c,j_c,l,pix
            INTEGER :: my_unit
            coefs(0,:)= (/ 1, 2, 3, 2, 1 /)
            coefs(1,:)= (/ 2, 8, 12, 8, 2 /)
            coefs(2,:)= (/ 3, 12, 16, 12, 3 /)
            coefs(3,:)= (/ 2, 8, 12, 8, 2 /)
            coefs(4,:)= (/ 1, 2, 3, 2, 1 /)

            DO p=1, passes
            DO j=2,cols-3
                DO i=2,rows-3
                    DO l=0,2
                        pix = 0
                        DO i_c=0,4
                            DO j_c=0,4
                                pix = pix + pic((i+i_c-2)*3*cols+(j+j_c-2)*3+l)*(coefs(i_c,j_c)) 
                            END DO
                        END DO
                        blurred(i*3*cols+j*3+l)=pix/128
                    END DO
                END DO
            END DO

            END DO
        END SUBROUTINE blur

        SUBROUTINE out_pic(pic, name)
            INTEGER(kind=1),DIMENSION(0:),INTENT(IN) :: pic
            CHARACTER(len=*),INTENT(IN)              :: name  
            INTEGER                                  :: my_unit

            OPEN(NEWUNIT=my_unit, FILE=name, status='replace' ,ACCESS="stream", FORM="unformatted")
            WRITE(my_unit,POS=1) pic 
            CLOSE(my_unit)
        END SUBROUTINE out_pic

end module pixels

PROGRAM blur_pix
    use PIXELS
    implicit none

    integer                                      :: rows, cols, i, long, j, check, numarg
    integer                                      :: passes
    integer  (kind=1), dimension(:), allocatable :: pic,blurred_pic
    character(len=: ), allocatable               :: arg
    integer, dimension(2)                        :: n
 
    rows = 2232
    cols = 4000
    passes = 40

    write(6,"(a23,i7,a2,i7)") "Size of the picture is ",rows," x",cols
    allocate(pic(0:rows*3*cols), blurred_pic(0:rows*3*cols))

    call read_matrix_from_file("pic.rgb", pic, rows, cols)
    call blur(pic, blurred_pic, rows, cols, passes)

    call out_pic(pic, "pic.rgb")
    call out_pic(blurred_pic, "blurred.rgb")

    deallocate(pic, blurred_pic)
END PROGRAM blur_pix


In [ ]:
from idrcomp import compare_rgb
"""compare the original and blurred pictures.
It is possible to display a cropped version of the images for better visualization.
For example (0.0,1.0,0.0,1.0) will display the whole image.
and (0.5,1.0,0.5,1.0) will display the bottom right part of the pictures"""
compare_rgb("pic.rgb", "blurred.rgb", (0.0, 1.0, 0.0, 1.0), 2232, 4000)

### Solution

In [ ]:
import os
picture = os.path.join("..", "..", "pictures", "midris.jpg")
from idrcomp import convert_jpg_to_raw
convert_jpg_to_raw(picture, "pic.rgb")

Example stored in: `../../examples/Fortran/blur_simple_solution.f90`

In [ ]:
%%idrrun -a
MODULE pixels
    USE OPENACC
    IMPLICIT NONE
    CONTAINS
        SUBROUTINE read_matrix_from_file(FILENAME, PIC, ROWS, COLS)
            IMPLICIT NONE
            CHARACTER(LEN=*), INTENT(IN)                :: FILENAME
            INTEGER(kind=1), DIMENSION(0:), INTENT(OUT) :: PIC
            INTEGER, INTENT(IN)                         :: ROWS, COLS
        
            INTEGER                                     :: FILE_UNIT, I, TOTAL_SIZE, IO_STATUS
            INTEGER(KIND=8)                             :: READ_SIZE
            CHARACTER(LEN=80)                           :: IO_MSG
            OPEN(NEWUNIT=FILE_UNIT, FILE=FILENAME, FORM='UNFORMATTED', ACCESS='stream')
        
            TOTAL_SIZE = ROWS * COLS * 3
            READ(FILE_UNIT, IOSTAT=IO_STATUS, IOMSG=IO_MSG) PIC(0:TOTAL_SIZE-1)
        
            CLOSE(FILE_UNIT)
        END SUBROUTINE read_matrix_from_file  

        SUBROUTINE blur(pic, blurred, rows, cols, passes)
            INTEGER(kind=1),DIMENSION(0:),INTENT(IN)  :: pic
            INTEGER(kind=1),DIMENSION(0:),INTENT(OUT) :: blurred
            INTEGER,DIMENSION(0:4,0:4)                :: coefs
            INTEGER,INTENT(IN)                        :: rows,cols, passes
            INTEGER(kind=4)                           :: i,j,p,i_c,j_c,l,pix
            INTEGER :: my_unit
            coefs(0,:)= (/ 1, 2, 3, 2, 1 /)
            coefs(1,:)= (/ 2, 8, 12, 8, 2 /)
            coefs(2,:)= (/ 3, 12, 16, 12, 3 /)
            coefs(3,:)= (/ 2, 8, 12, 8, 2 /)
            coefs(4,:)= (/ 1, 2, 3, 2, 1 /)

            DO p=1, passes
            !$acc parallel loop copyin(pic(0:), coefs(0:4,0:4)) copyout(blurred(0:))
            DO j=2,cols-3
                DO i=2,rows-3
                    DO l=0,2
                        pix = 0
                        DO i_c=0,4
                            DO j_c=0,4
                                pix = pix + pic((i+i_c-2)*3*cols+(j+j_c-2)*3+l)*(coefs(i_c,j_c)) 
                            END DO
                        END DO
                        blurred(i*3*cols+j*3+l)=pix/128
                    END DO
                END DO
            END DO

            END DO
        END SUBROUTINE blur

        SUBROUTINE out_pic(pic, name)
            INTEGER(kind=1),DIMENSION(0:),INTENT(IN) :: pic
            CHARACTER(len=*),INTENT(IN)              :: name  
            INTEGER                                  :: my_unit

            OPEN(NEWUNIT=my_unit, FILE=name, status='replace' ,ACCESS="stream", FORM="unformatted")
            WRITE(my_unit,POS=1) pic 
            CLOSE(my_unit)
        END SUBROUTINE out_pic

end module pixels

PROGRAM blur_pix
    use PIXELS
    implicit none

    integer                                      :: rows, cols, i, long, j, check, numarg
    integer                                      :: passes
    integer  (kind=1), dimension(:), allocatable :: pic,blurred_pic
    character(len=: ), allocatable               :: arg
    integer, dimension(2)                        :: n
 
    rows = 2232
    cols = 4000
    passes = 40

    write(6,"(a23,i7,a2,i7)") "Size of the picture is ",rows," x",cols
    allocate(pic(0:rows*3*cols), blurred_pic(0:rows*3*cols))

    call read_matrix_from_file("pic.rgb", pic, rows, cols)
    call blur(pic, blurred_pic, rows, cols, passes)

    call out_pic(pic, "pic.rgb")
    call out_pic(blurred_pic, "blurred.rgb")

    deallocate(pic, blurred_pic)
END PROGRAM blur_pix

Now we can compare the original picture with its blurred version:

In [ ]:
from idrcomp import compare_rgb
"""compare the original and blurred pictures.
It is possible to display a cropped version of the images for better visualization.
For example (0.0,1.0,0.0,1.0) will display the whole image.
and (0.5,1.0,0.5,1.0) will display the bottom right part of the pictures"""
compare_rgb("pic.rgb", "blurred.rgb", (0.0, 1.0, 0.0, 1.0), 2232, 4000)

## Reductions with OpenACC

Your code is performing a reduction when a loop is updating at each cycle the same variable:

For example, if you perform the sum of all elements in an array:

```fortran
do i = 1, size_array
    summation = summation + array(i)
enddo
```


If you run your code sequentially no problems occur.
However we are here to use a massively parallel device to accelerate the computation.

In this case we have to be careful since simultaneous read/write operations can be performed on the same variable.
The result is not sure anymore because we have a race condition.

For some operations, OpenACC offers an efficient mechanism if you use the _reduction(operation:var1,var2,...)_ clause which is available for the directives:
- `!$acc loop reduction(op:var1)` 
- `!$acc parallel reduction(op:var1)` 
- `!$acc kernels reduction(op:var1)` 
- `!$acc serial reduction(op:var1)`

__Important__: Please note that for a lot of cases, the NVIDIA compiler (formerly PGI) is able to detect that a reduction is needed and will add it implicitly.
We advise you make explicit all implicit operations for code readability/maintenance.

### Available operations

The set of operations is limited. We give here the most common:

| Operator   | Operation    | Syntax                     |
|------------|--------------|----------------------------|
| +          | sum          | `reduction(+:var1, ...)`   |
| *          | product      | `reduction(*:var2, ...)`   |
| max        | find maximum | `reduction(max:var3, ...)` |
| min        | find minimum | `reduction(min:var4,...)`  |

Other operators are available, please refer to the OpenACC specification for a complete list.

#### Reduction on several variables

If you perform a reduction with the same operation on several variables then you can give a comma separated list after the colon:
```fortran
!$acc parallel loop reduction(+:var1, var2,...)
```


If you perform reductions with different operators then you have to specify a _reduction_ clause for each operator:
```fortran
!$acc parallel reduction(+:var1, var2) reduction(max:var3) reduction(*: var4)
```

### Exercise

Let's do some statistics on the exponential function.
The goal is to compute

- the integral of the function between 0 and $\pi$ using the trapezoidal method
- the maximum value
- the minimum value

You have to:

- Run the following example on the CPU. How much time does it take to run?
- Add the directives necessary to create one kernel for the loop that will run on the GPU
- Run the computation on the GPU. How much time does it take?

Your solution is considered correct if no implicit operation is reported by the compiler.

Example stored in: `../../examples/Fortran/reduction_exponential_exercise.f90`

In [ ]:
%%idrrun -a
program reduction_exponential
    use iso_fortran_env, only : INT32, REAL64
    implicit none
    ! current position and values
    real   (kind=REAL64) :: x, y, x_p
    real   (kind=REAL64) :: double_min, double_max, begin, fortran_pi, summation
    real   (kind=REAL64) :: step_l
    ! number of division of the function
    integer(kind=INT32 ) :: nsteps 
    integer(kind=INT32 ) :: i

    nsteps     = 1e9
    begin      = 0.0_real64                  ! x min 
    fortran_pi = acos(-1.0_real64)           ! x max
    summation  = 0.0_real64                  ! sum of elements
    step_l     = (fortran_pi - begin)/nsteps ! length of the step

    double_min = huge(double_min)
    double_max = tiny(double_max)

    do i = 1, nsteps
        x   =  i * step_l
        x_p = (i+1) * step_l
        y   = (exp(x)+exp(x_p))*0.5_real64
        summation = summation + y
        if (y .lt. double_min) double_min = y
        if (y .gt. double_max) double_max = y
    enddo

    ! print the stats
    write(0,"(a38,f20.8)") "The MINimum value of the function is: ",double_min
    write(0,"(a38,f20.8)") "The MAXimum value of the function is: ",double_max
    write(0,"(a33,f3.1,a1,f8.6,a6,f20.8)") "The integral of the function on [", &
                                            begin,",",fortran_pi,"] is: ",summation*step_l
    write(0,"(a18,es20.8)") "   difference is: ",exp(fortran_pi)-exp(begin)-summation*step_l
end program reduction_exponential

#### Solution

Example stored in: `../../examples/Fortran/reduction_exponential_solution.f90`

In [ ]:
%%idrrun -a
program reduction_exponential
    use iso_fortran_env, only : INT32, REAL64
    implicit none
    ! current position and values
    real   (kind=REAL64) :: x, y, x_p
    real   (kind=REAL64) :: double_min, double_max, begin, fortran_pi, summation
    real   (kind=REAL64) :: step_l
    ! number of division of the function
    integer(kind=INT32 ) :: nsteps 
    integer(kind=INT32 ) :: i

    nsteps     = 1e9
    begin      = 0.0_real64                  ! x min 
    fortran_pi = acos(-1.0_real64)           ! x max
    summation  = 0.0_real64                  ! sum of elements
    step_l     = (fortran_pi - begin)/nsteps ! length of the step

    double_min = huge(double_min)
    double_max = tiny(double_max)
    !$acc parallel loop reduction(+:summation) reduction(min:double_min) reduction(max:double_max)
    do i = 1, nsteps
        x   =  i * step_l
        x_p = (i+1) * step_l
        y   = (exp(x)+exp(x_p))*0.5_real64
        summation = summation + y
        if (y .lt. double_min) double_min = y
        if (y .gt. double_max) double_max = y
    enddo

    ! print the stats
    write(0,"(a38,f20.8)") "The MINimum value of the function is: ",double_min
    write(0,"(a38,f20.8)") "The MAXimum value of the function is: ",double_max
    write(0,"(a33,f3.1,a1,f8.6,a6,f20.8)") "The integral of the function on [",begin, &
                                           ",",fortran_pi,"] is: ",summation*step_l
    write(0,"(a18,es20.8)") "   difference is: ",exp(fortran_pi)-exp(begin)-summation*step_l
end program reduction_exponential

### Important Notes

- A special kernel is created for reduction. With NVIDIA compiler its name is the name of the "parent" kernel with \_red appended.
- You may want to use other directives to "emulate" the behavior of a reduction (it is possible by using _atomic_ operations).
  We strongly discourage you from doing this. The _reduction_ clause is much more efficient.